In [3]:
import pandas as pd
import glob
import numpy as np

In [24]:
def updateDataIndex():
    scheds = glob.glob("schedules/*.csv")
    pools = glob.glob("pools/*.csv")
    s_len = []
    Schedule = []
    for sched in sorted(scheds):
        s_len.append(len(pd.read_csv(sched)))
        Schedule.append(sched.split('/')[1].split('_')[1].split('.')[0])
    p_len = []
    Pool = []
    for pool in sorted(pools):
        p_len.append(len(pd.read_csv(sched)))
        Pool.append(pool.split('/')[1].split('_')[1].split('.')[0])

    Pool += ['nan'] * (len(Schedule) - len(Pool))
    p_len += ['nan'] * (len(s_len) - len(p_len))

    data = {'Schedule':Schedule,'shifts':s_len,'Pool':Pool,'employees':p_len}

    di = pd.DataFrame(data)
    
    return di

In [26]:
di = updateDataIndex()
di.to_csv('data_index.csv',index=False)

In [28]:
def problemIndex():
    di = updateDataIndex()
    #pd.read_csv('data_index.csv',dtype=str)
    di['shifts'] = di['shifts'].astype(int) 
    
    pi = pd.DataFrame(columns=['Schedule', 'Pool'])

    scheds = di.Schedule.to_list()
    pools = di.Pool.to_list()
    pools = [x for x in pools if str(x) != 'nan']

    for sched in scheds:
        for pool in pools:
            pi.loc[0 if pd.isnull(pi.index.max()) else pi.index.max() + 1] = [sched] + [pool]

    pi = pd.merge(pi, di[['Schedule', 'shifts']], on ='Schedule', how ='left')
    pi = pd.merge(pi, di[['Pool', 'employees']], on ='Pool', how ='left')
    pi['employees'] = pi['employees'].astype(int)
    pi['Nodes'] = pi['shifts'] + pi['employees']
    pi['Ratio'] = pi['shifts'] / pi['employees']
    
    return pi


In [29]:
problemIndex()
#pi.to_csv('problem_index.csv', index=False)

,Schedule,Pool,shifts,employees,Nodes,Ratio
0,0001,0001,4,8,12,0.5
1,0001,0002,4,8,12,0.5
2,0001,0003,4,8,12,0.5
3,0001,0004,4,8,12,0.5
4,0001,0005,4,8,12,0.5
...,...,...,...,...,...,...
79,0012,0003,8,8,16,1.0
80,0012,0004,8,8,16,1.0
81,0012,0005,8,8,16,1.0
82,0012,0006,8,8,16,1.0


In [2]:
# ON HOLD

# check if values are as expected   
    # sequential shift IDs
    # proper case spelling
# check that size is as expected

In [ ]:
scheds = glob.glob("schedules/*.csv")
pools = glob.glob("pools/*.csv")

In [ ]:
shift_day_of_week = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
shift_type = ['Morning','Evening']


In [ ]:
errorDict = {}
for sched in scheds:
        id = sched.split('/')[1].split('_')[1].split('.')[0]
        s = pd.read_csv(sched,dtype={'shift_id':'str'})
        
        #stateDict[id] = 
        

In [ ]:
for pool in pools:
    id = pool.split('/')[1].split('_')[1].split('.')[0]
    p = pd.read_csv(pool,dtype={'employee_id':'str'})
    #stateDict[id] = 

In [ ]:
# convert all to matrices
# check for unqiueness

In [6]:
def matrify(pool, schedule):
    schedule = pd.get_dummies(schedule,drop_first=True)
    #shift_features = schedule.shape[1]
    for i in pd.get_dummies(pool).columns.to_list():
        schedule[i] = 0

    return schedule.to_numpy()

In [8]:
def problemValidation():
    scheds = glob.glob("schedules/*.csv")
    pools = glob.glob("pools/*.csv")

    stateDict = {}
    for sched in scheds:
        for pool in pools:
            id = sched.split('/')[1].split('_')[1].split('.')[0]+pool.split('/')[1].split('_')[1].split('.')[0]
            s = pd.read_csv(sched,dtype={'shift_id':'str'})
            p = pd.read_csv(pool,dtype={'employee_id':'str'})
            stateDict[id] = matrify(p, s)

    dupes=[]
    for key, value in stateDict.items(): 
        for k, v in stateDict.items(): 
            if key != k:
                if np.array_equal(value, v):
                    dupes.append((key,k))

    print(f"{len(dupes)} duplicate problems found in data set.")

    return dupes

In [11]:
dupes =  problemValidation()

0 duplicate problems found in data set.


In [ ]:
### problem loader 

In [36]:
def problemLoader(max_shifts):
    pi = problemIndex()
    pi.head()

    selectedProbs = pi[pi['shifts'] <=max_shifts] 
    selectedProbs



    glob_list = []
    for i in range(len(selectedProbs)):
        s = selectedProbs['Schedule'].iloc[i]
        p = selectedProbs['Pool'].iloc[i]
        glob_list.append((s,p))

    return glob_list

#problemLoader(max_shifts=5)